<a href="https://colab.research.google.com/github/wincowgerDEV/OpenSpecyAI/blob/main/Deep_Learning_Classification_of_HDPE_and_PET_Spectra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Classification of HDPE and PET Spectra
---



### Aim:
    This project aims to develop a Convolution Network to perform identification of HDPE and PET FTIR spectra.
    
    Tensorflow and Keras APIs were used for the development of a 1D Sequential CNN of 7 Layers.
    
    The code was adapted for spectral analysis using a model developed for accelerometer data:https://github.com/bharatm11/1D_CNN_Human_activity_recognition/blob/master/Deep_Learning_for_Human_Activity_Recognition.ipynb

    The dataset comes from Chabuka et al. 2020 https://doi.org/10.1177%2F0003702820923993 and was augmented with additional noise and baseline shift. The test data can be downloaded from GitHub.
    
     

Getting Started

Download the training data from: https://osf.io/bes7h/

Put the data into a known location on you Google drive. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import metrics
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Conv1D, MaxPooling1D,GlobalAveragePooling1D,GlobalAvgPool1D, Reshape, Activation
from keras import optimizers
import keras
from keras.utils import np_utils
#%matplotlib inline
plt.style.use('ggplot')
from keras.optimizers import SGD


### Pipeline

The training process starts by reading the data and normalizing it. This normalized data is then segmented into slices of window size 1300 which translates to individual spectra. These chunks are then randomly split into training and test sets. For the results shown in this report, 70% data as taken into the test set and the remaining was used in the test set for validation of the training algorithm. This training data was fed to a 1D CNN network which is described below.




In [ ]:
'''***********************'''
#DEFINE NETWORK PARAMETERS
trainSplitRatio = 0.7 # split ratio for test and validation
window_size = 1300 #Length of slice.
numFilters1 = 100 # number of filters in first Conv1D layer
kernalSize = 10 # kernal size of the Conv2D layer
batchSize = 10
numNueronsFCL2 = 160 # number of filters in fully connected output layer 
dropout = 0.5 #dropout rate. % of neurons converted to 0 weight before softmax
epochs = 50
'''***********************'''

'***********************'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#DEFINE HELPER FUNCTIONS
def read_data(file_path):
    print("reading data")
    column_names = ['Wavelength','Absorbance','Polymer', 'SpectrumID']
    data = pd.read_csv(file_path,header = None, names = column_names)
    print("finished reading data")
    return data

def feature_normalize(dataset):
    mu = np.mean(dataset,axis = 0)
    sigma = np.std(dataset,axis = 0)
    return (dataset - mu)/sigma


def windows(data, size):
    start = 0
    while start < data.count():
        yield int(start), int(start + size)
        start += (size / 2)

def segment_signal(data,window_size):
    segments = np.empty((0,window_size,2))
    labels = np.empty((0))
    for (start, end) in windows(data["Wavelength"], window_size):
        #x = data["x-axis"][start:end]
        #y = data["y-axis"][start:end]
        #z = data["z-axis"][start:end]
        x = data["Wavelength"][start:end]
        y = data["Absorbance"][start:end]
        if(len(dataset["Wavelength"][start:end]) == window_size):
            segments = np.vstack([segments,np.dstack([x,y])])#,z])])
            labels = np.append(labels,stats.mode(data["Polymer"][start:end])[0][0]) #activity
    return segments, labels
#READ AND NORMALIZE DATA
dataset = read_data('/content/gdrive/My Drive/GrayLab/Projects/Plastics/ActiveProjects/OpenSpecy/Data/Processed Data/DataAugmentation/ChabukaAugmentedDataProcessed.txt') #You will need to change this path to the location that you place the file in your own google drive that you link to.
dataset.dropna(axis=0, how='any', inplace= True)
print("normalizing x")
dataset['Wavelength'] = feature_normalize(dataset['Wavelength'])
print("normalizing y")
dataset['Absorbance'] = feature_normalize(dataset['Absorbance'])
#print("normalizing z")
#dataset['z-axis'] = feature_normalize(dataset['z-axis'])

reading data
finished reading data
normalizing x
normalizing y


In [ ]:
dataset

,Wavelength,Absorbance,Polymer,SpectrumID
0,-1.730573,2.677583,HDPE,HDPESample 1Beauty C
1,-1.727908,2.670636,HDPE,HDPESample 1Beauty C
2,-1.725244,2.673689,HDPE,HDPESample 1Beauty C
3,-1.722579,2.674474,HDPE,HDPESample 1Beauty C
4,-1.719914,2.672644,HDPE,HDPESample 1Beauty C
...,...,...,...,...
3889595,1.710435,0.403935,PET,PETSample 135Beauty C10
3889596,1.713099,0.339660,PET,PETSample 135Beauty C10
3889597,1.715764,0.201673,PET,PETSample 135Beauty C10
3889598,1.718428,0.559599,PET,PETSample 135Beauty C10


### This section plots one window size long plots for each class of the normalized data

In [ ]:
#SEGMENT DATA, LABELS INTO WINDOW_SIZE
print("segmenting data into windows")
segments, labels = segment_signal(dataset,window_size)
labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
reshaped_segments = segments.reshape(len(segments), 1,window_size, 2)
print("segmented data in windows")
#SPLIT DATA INTO TEST AND TRAINING SETS
print("Splitting data into test and training sets")
train_test_split = np.random.rand(len(reshaped_segments)) < trainSplitRatio
train_x = reshaped_segments[train_test_split]
train_y = labels[train_test_split]
test_x = reshaped_segments[~train_test_split]
test_y = labels[~train_test_split]
print("Ready for training")

segmenting data into windows
segmented data in windows
Splitting data into test and training sets
Ready for training


In [ ]:
segments[100]

array([[-1.73057311,  3.1828699 ],
       [-1.72790835,  3.17505994],
       [-1.72524387,  3.17379161],
       ...,
       [ 1.72543644,  2.67631326],
       [ 1.72810037,  2.67746493],
       [ 1.73076568,  2.67732151]])

In [ ]:
labels[100]

array([1, 0], dtype=int8)

In [ ]:
train_x

array([[[[-1.73057311e+00,  2.81590240e+00],
         [-1.72790835e+00,  2.81215857e+00],
         [-1.72524387e+00,  2.80310570e+00],
         ...,
         [ 1.72543644e+00,  2.53924801e+00],
         [ 1.72810037e+00,  2.53982392e+00],
         [ 1.73076568e+00,  2.53998005e+00]]],


       [[[ 1.42797108e-03,  2.38835894e+00],
         [ 4.09328196e-03,  2.38933465e+00],
         [ 6.75721113e-03,  2.38772291e+00],
         ...,
         [-6.56519815e-03,  2.46968322e+00],
         [-3.90126898e-03,  2.46792369e+00],
         [-1.23595810e-03,  2.46758476e+00]]],


       [[[-1.73057311e+00,  2.62004893e+00],
         [-1.72790835e+00,  2.61570875e+00],
         [-1.72524387e+00,  2.61926459e+00],
         ...,
         [ 1.72543644e+00,  2.46896444e+00],
         [ 1.72810037e+00,  2.47012871e+00],
         [ 1.73076568e+00,  2.47145679e+00]]],


       ...,


       [[[-1.73278384e+00, -1.48752967e-02],
         [-1.73011908e+00,  1.09328066e-01],
         [-1.72745460e+00, -2.04

In [ ]:
train_y

array([[0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       ...,
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0]], dtype=int8)

In [ ]:
#EXTRACT DATASET PARAMETERS
numOfRows = segments.shape[1]
print(numOfRows)
numOfColumns = segments.shape[2]
print(numOfColumns)
print(train_x.shape[2])
print(train_y.shape[1])
num_classes = labels.shape[1]
num_data_parameters = train_x.shape[3]
input_shape = window_size*num_data_parameters

1300
2
1300
2


### CNN Network

A 1D CNN network was used considering the dimensions of the data. Each row of the data consists of the corresponding Wavelength and Absorbance from the spectrum and the height of the layer determines the number of instances of data equalling the window size which is 1300 in our case. Only the size of the input and output layers needs to be specified explicitly. The network estimates the size of the hidden layers on it's own.

The network used here is of sequential type which means that it's basically a stack of layers. These layers include:
* Input layer
* First 1D CNN Layer
* A max pooling layer
* Second 1D CNN Layer 
* An average pooling layer
* A dropout layer
* A fully connected Softmax Activated layer

**Input Layer:** The input data consists of 1300 spectral point slices long instances of the spectrum. Hence, the size of the input layer needs to be reshaped to 1300x2. The data passes through the input layer as a vector. The output for this layer is 1300x2.

**First 1D CNN Layer:** This defines a filter of kernel size 10. 100 such filters are defined in this layer to enable it to learn 100 different features.

**A max pooling layer:** This is used to reduce the complexity of the output and to prevent overfitting of the data. Using a pooling layer size of 3 reduces the size of the output matrix to 1/3rd of the input matrix.

**Second 1D CNN Layer:** This layer enables the network to pick up higher level features which were missed in the First CNN layer. 

**Average pooling layer:** This averages the value of two weights in the network thereby further reducing overfitting. 

**Dropout layer:** This randomly assignms a weight of 0 to the neurons in the network. A value of 0.5 indicates that 50% of the neurons turn 0.

**Fully connected Softmax Activated layer:** This reduces the output to the desired height of 6 which indicates the number of activity classes in the data. Softmax forces all six outputs of the neural network to sum up to one.

In [ ]:
#DEFINE CNN MODEL
# 1D CNN neural network

model_m = Sequential()
model_m.add(Reshape((window_size, num_data_parameters), input_shape=(1,numOfRows,numOfColumns)))
model_m.add(Conv1D(numFilters1, kernalSize, activation='relu', input_shape=(window_size, num_data_parameters)))
model_m.add(MaxPooling1D(3))
model_m.add(Conv1D(numNueronsFCL2, 10, activation='relu'))
model_m.add(GlobalAveragePooling1D())

model_m.add(Dropout(dropout))

model_m.add(Dense(num_classes, activation='softmax'))
print(model_m.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 1300, 2)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1291, 100)         2100      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 430, 100)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 421, 160)          160160    
_________________________________________________________________
global_average_pooling1d (Gl (None, 160)               0         
_________________________________________________________________
dropout (Dropout)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 3

In [ ]:
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]

model_m.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

BATCH_SIZE = 400
EPOCHS = epochs

history = model_m.fit(train_x,
                      train_y,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)

Epoch 1/50
9/9 [==============================] - 32s 4s/step - loss: 0.6931 - accuracy: 0.5397 - val_loss: 0.6734 - val_accuracy: 0.6066
Epoch 2/50
9/9 [==============================] - 32s 4s/step - loss: 0.6458 - accuracy: 0.6486 - val_loss: 0.6262 - val_accuracy: 0.7190
Epoch 3/50
9/9 [==============================] - 34s 4s/step - loss: 0.6105 - accuracy: 0.6963 - val_loss: 0.6008 - val_accuracy: 0.7237
Epoch 4/50
9/9 [==============================] - 32s 4s/step - loss: 0.5764 - accuracy: 0.7174 - val_loss: 0.5720 - val_accuracy: 0.7178
Epoch 5/50
9/9 [==============================] - 32s 4s/step - loss: 0.5580 - accuracy: 0.7280 - val_loss: 0.5788 - val_accuracy: 0.7272
Epoch 6/50
9/9 [==============================] - 32s 4s/step - loss: 0.5400 - accuracy: 0.7385 - val_loss: 0.5703 - val_accuracy: 0.7330
Epoch 7/50
9/9 [==============================] - 32s 4s/step - loss: 0.5310 - accuracy: 0.7397 - val_loss: 0.5434 - val_accuracy: 0.7237
Epoch 8/50
9/9 [==================

In [1]:
score = model_m.evaluate(test_x, test_y,batch_size=BATCH_SIZE, verbose=2)
print("The test accuracy is",score[1]*100,"%")

NameError: ignored

### Results

The network was succesfully trained to recognize the difference between HDPE and PP. 

The model has a test accuracy of ~85%. 